In [7]:
from ctypes import *
import math
import random
import cv2

In [8]:
pathYolo = "/media/feliz/Safira/GitHub/Gridkey/Yolo/darknet/"
pathImage = pathYolo +  "data/dog.jpg"
pathVedio = "/media/feliz/Safira/GitHub/Gridkey/Car-Detection/Testing/Videos/3.avi"


In [9]:
def sample(probs):
    s = sum(probs)
    probs = [a/s for a in probs]
    r = random.uniform(0, 1)
    for i in range(len(probs)):
        r = r - probs[i]
        if r <= 0:
            return i
    return len(probs)-1

def c_array(ctype, values):
    return (ctype * len(values))(*values)

class NODE(Structure):
    pass

NODE._fields_ = [("val", c_void_p),
                ("next", POINTER(NODE)),
                ("prev", POINTER(NODE))]  
# https://docs.python.org/2/library/ctypes.html#incomplete-types

class LIST(Structure):
     _fields_ = [("size", c_int),
                ("front", POINTER(NODE)),
                ("back", POINTER(NODE))] 

class BOX(Structure):
    _fields_ = [("x", c_float),
                ("y", c_float),
                ("w", c_float),
                ("h", c_float)]

class IMAGE(Structure):
    _fields_ = [("w", c_int),
                ("h", c_int),
                ("c", c_int),
                ("data", POINTER(c_float))]

class METADATA(Structure):
    _fields_ = [("classes", c_int),
                ("names", POINTER(c_char_p))]


In [10]:
class Darknet():
    #     Shared Variables
    lib = CDLL(pathYolo + "libdarknet.so", RTLD_GLOBAL)
#     lib.network_width.argtypes = [c_void_p]
#     lib.network_width.restype = c_int
#     lib.network_height.argtypes = [c_void_p]
#     lib.network_height.restype = c_int

#     predict = lib.network_predict_p
#     predict.argtypes = [c_void_p, POINTER(c_float)]
#     predict.restype = POINTER(c_float)

#     detect = lib.network_predict_p
#     detect.argtypes = [c_void_p, IMAGE, c_float, c_float, c_float, POINTER(BOX), POINTER(POINTER(c_float))]

#     reset_rnn = lib.reset_rnn
#     reset_rnn.argtypes = [c_void_p]

#     letterbox_image = lib.letterbox_image
#     letterbox_image.argtypes = [IMAGE, c_int, c_int]
#     letterbox_image.restype = IMAGE

    #   Common Variables
    load_net = lib.load_network_p
    load_net.argtypes = [c_char_p, c_char_p, c_int]
    load_net.restype = c_void_p
#   network *load_network_p(char *cfg, char *weights, int clear)

    load_meta = lib.get_metadata
    lib.get_metadata.argtypes = [c_char_p]
    lib.get_metadata.restype = METADATA
#   metadata get_metadata(char *file)

    load_image = lib.load_image_color
    load_image.argtypes = [c_char_p, c_int, c_int]
    load_image.restype = IMAGE
#   image load_image_color(char *filename, int w, int h)


    #     Classify
    predict_image = lib.network_predict_image
    predict_image.argtypes = [c_void_p, IMAGE]
    predict_image.restype = POINTER(c_float)


    #     Network/Border Detect
    make_boxes = lib.make_boxes
    make_boxes.argtypes = [c_void_p]
    make_boxes.restype = POINTER(BOX)
#   box *make_boxes(network *net)

    make_probs = lib.make_probs
    make_probs.argtypes = [c_void_p]
    make_probs.restype = POINTER(POINTER(c_float))
#   float **make_probs(network *net)
    
    num_boxes = lib.num_boxes
    num_boxes.argtypes = [c_void_p]
    num_boxes.restype = c_int
#   int num_boxes(network *net)
    
    free_image = lib.free_image
    free_image.argtypes = [IMAGE]   
#   void free_image(image m)
    
    free_ptrs = lib.free_ptrs
    free_ptrs.argtypes = [POINTER(c_void_p), c_int]
#   void free_ptrs(void **ptrs, int n)

    network_detect = lib.network_detect
    network_detect.argtypes = [c_void_p, IMAGE, c_float, c_float, c_float, POINTER(BOX), POINTER(POINTER(c_float))]
#   void network_detect(network *net, image im, float thresh, float hier_thresh, float nms, box *boxes, float **probs)
    
    #    Detector For Vedio (Only considering Arguments for vedio)
    read_data_cfg = lib.read_data_cfg
    read_data_cfg.argtypes = [c_char_p]
    read_data_cfg.restype = POINTER(LIST)
    
    option_find_int = lib.option_find_int
    option_find_int.argtypes = [POINTER(LIST), c_char_p, c_int]
    option_find_int.restype = c_int
    
    option_find_str = lib.option_find_str
    option_find_str.argtypes = [POINTER(LIST), c_char_p, c_char_p]
    option_find_str.restype = c_char_p
    
    get_labels = lib.get_labels
    get_labels.argtypes = [c_char_p]
    get_labels.restype = POINTER(c_char_p)
    
    demo = lib.demo
    demo.argtypes = [c_char_p,  c_char_p, c_float, c_int, c_char_p, POINTER(c_char_p), c_int, c_int, c_char_p, c_int, c_float, c_int, c_int, c_int, c_int] 
#   void demo(char *cfgfile, char *weightfile, float thresh, int cam_index, const char *filename, char **names, int classes, int delay, char *prefix, int avg_frames, float hier, int w, int h, int frames, int fullscreen)
    
    
    def __init__(self):
        pass
    
    def runDetector(self, cfgdata, cfg, weights = '0', filename = '0', prefix = '0',thresh = 0.24, hier_thresh = 0.5, camIndex = 0, frameSkip = 0, avg = 3, fps = 0, height = 0, width = 0 , fullscreen = 0, clear = 0):
        """
        This function converted from run_detector of darknet. Works for Video only
        Converts each frame to .jpg Image file after reading from vedio. 
        Needs more work to get it generate the complete thing as vedio.
        The jpg files are created in the directory where this notebook is present 
        """
        if weights == '0':
            print "Weights not specified"
            return 1
        if filename == '0':
            print "Input File to process not specified"
            return 1
        options = self.read_data_cfg(cfgdata)
        classes = self.option_find_int(options, "classes", 20)
        name_list = self.option_find_str(options, "names", pathYolo + "data/name.list")
        names = self.get_labels(name_list)
        self.demo(cfg,weights,thresh, camIndex, filename, names, classes, frameSkip, prefix, avg, hier_thresh, width, height, fps, fullscreen)
        
        
    
    def classify(self,net, meta, im):
        out = self.predict_image(net, im)
        res = []
        for i in range(meta.classes):
            res.append((meta.names[i], out[i]))
        res = sorted(res, key=lambda x: -x[1])
        return res
    

    def borderDetect(self,net, meta, image, thresh=.5, hier_thresh=.5, nms=.45):
        """
        This function gives the class, probability and its boundary co-ordinates 
        for the detected object in a Picture
        """
        im = self.load_image(image, 0, 0)
        boxes = self.make_boxes(net)
        probs = self.make_probs(net)
        num =   self.num_boxes(net)
        self.network_detect(net, im, thresh, hier_thresh, nms, boxes, probs)
        res = []
        for j in range(num):
            for i in range(meta.classes):
                if probs[j][i] > 0:
                    res.append((meta.names[i], probs[j][i], (boxes[j].x, boxes[j].y, boxes[j].w, boxes[j].h)))
        res = sorted(res, key=lambda x: -x[1])
        self.free_image(im)
        self.free_ptrs(cast(probs, POINTER(c_void_p)), num)
        return res  


In [11]:
if __name__ == "__main__":
    #    TINY YOLO 
    cfgdata = pathYolo +  "cfg/voc.data"
    cfg = pathYolo + "cfg/tiny-yolo-voc.cfg"
    weights = pathYolo + "weights/tiny-yolo-voc.weights"
    
#     YOLO 
#     cfgdata = pathYolo +  "cfg/coco.data"
#     cfg = pathYolo + "cfg/yolo.cfg"
#     weights = pathYolo + "weights/yolo.weights"
    
    d = Darknet()

#     Network/Boundary Detection for Objects
#     net = d.load_net(cfg,weights, 0)
#     meta = d.load_meta(cfgdata)
#     r = d.borderDetect(net, meta, pathImage)
#     print r

    #    Detector Vedio 
    d.runDetector(cfgdata,cfg,weights,pathVedio)

AttributeError: 'LP_c_char_p' object has no attribute 'value'

In [4]:
# cv2.startWindowThread()
# img = cv2.imread(pathImage)
# # cv2.rectangle(img,(int(226.77825927734375),int(376.65716552734375+289.3421325683594)),(int(226.77825927734375+189.18458557128906),int(376.65716552734375)),(0,255,0),3)
# cv2.imshow("Image",img)
# cv2.waitKey(27)


-1

In [5]:
# cv2.waitKey(0)
# cv2.destroyAllWindows()